Multi Armed Bandit
=============
- 손잡이가 n개인 슬롯머신
- 시간의 흐름에 따라 높은 보상을 주는 손잡이를 찾고
- 그 손잡이에서 오는 보상을 최대화 하는 문제

## 강화학습의 3가지 요소
1. 액션 의존성 - 각 액션은 다른 보상을 가져온다
2. 시간 의존성 - 보상은 시간이 지난 뒤에 주어지고, 드물게(sparse) 주어진다
3. 상태 의존성 - 보상은 환경의 상태에 좌우된다

#### 정책 (Policy)
- 한 액션이 어떤 보상과 관련이 있는지,
- 그래서 에이전트가 최적의 액션을 선택하도록 하는 것
- 시간이 지난 후 최대의 보상을 얻는 정책이 바로 최적의 정책

## 정책 경사
- NN은 일련의 가중치로 구성되는데,
- MAB문제는 이를 각각의 손잡이에 매치되어 결정된다
- 가중치를 초기화 할 때 모두 1로 하면 모든 보상에 대해 낙관적으로 시작한다는 의미가 된다

#### 예) Greedy Policy
- 네트워크 업데이트 시 단순히 e의 확률로 랜덤 액션을 취하는 policy(손잡이)를 정했다고 가정
- 에이전트는 기본적으로 최댓값을 예상하는 액션을 선택
- 하지만 랜덤(e)의 확률로 다른 액션을 취함
- 이렇게 여러 시도를 해보면서 학습을 진행하게 되고, 보상은 1 또는 -1로 받게된다
- 이 비용을 수식(정책 비용 수식, Policy Loss Equation)으로 표현하면: 

                                   Loss = -log(pi) * A

#### 수식 설명
- A는 어드밴티지(Advantage) - 현재 취하는 액션이 미리 정한 기준선보다 얼마나 나은지의 정도
    - 이는 결국 에이전트가 각 액션에 대해 받는 보상
- pi는 정책(Policy) - 선택한 액션의 가중치
- 비용 함수 (Loss Function) - (+)의 보상에 대해 가중치 증가, (-)의 보상에 대해 가중치 감소의 목적

## 코드예제

In [1]:
#package imports
import numpy as np
import tensorflow as tf
import tensorflow.contrib.slim as slim

### 밴딧

In [9]:
#손잡이가 4개인 밴딧
bandit_arms = [0.2, 0, -0.2, -2]
num_arms = len(bandit_arms)

#랜덤한 정규분포를 생성하고, bandit보다 크면 1 아니면 -1을 반환
def pullBandit(bandit):
    #랜덤한 값을 구함
    result = np.random.randn(1)
    
    if result > bandit: #(+)보상 반환
        return 1 
    else: #(-)보상 반환
        return -1 


### 에이전트

In [10]:
tf.reset_default_graph()

#NN의 feed-forward를 구현
#값을 1로 시작 (1이면 모든 보상에 대해 낙관적으로 시작한다는 의미)
weights = tf.Variable(tf.ones([num_arms]))
output = tf.nn.softmax(weights)

# 학습 과정을 구현
# 보상과 선택된 액션을 NN에 feed --> 비용을 계산
# 비용을 사용해 NN을 업데이트
reward_holder = tf.placeholder(shape=[1], dtype=tf.float32)
action_holder = tf.placeholder(shape=[1], dtype=tf.int32)

responsible_output = tf.slice(output, action_holder, [1])
# 위 예제에서 설정한 Loss함수 (Loss = -log(pi) * A)
loss = -(tf.log(responsible_output) * reward_holder)
# Adam을 사용하고 학습률을 0.001로 설정
optimizer = tf.train.AdamOptimizer(learning_rate=1e-3)
update = optimizer.minimize(loss)

### 에이전트 학습

In [12]:
# 학습시킬 에피소드의 수
total_episodes = 1000
# 각 밴딧에 대한 점수판, 0으로 시작
total_reward = np.zeros(num_arms)

init = tf.global_variables_initializer()

#텐서플로 그래프를 론칭
with tf.Session() as sess:
    sess.run(init)
    i = 0
    while i < total_episodes:
        #볼츠만 분포에 따라 액션 선택
        actions = sess.run(output)
        a = np.random.choice(actions, p=actions)
        action = np.argmax(actions == a)
        
        #밴딧 중 하나를 선택해 보상을 취함
        reward = pullBandit(bandit_arms[action])
        
        # NN의 weight(가중치)를 업데이트
        _, resp, ww = sess.run([update, responsible_output, weights], 
                               feed_dict={reward_holder: [reward], action_holder: [action]})
        
        # 보상의 합계 업데이트
        total_reward[action] += reward
        
        
        if i % 30 == 0:
            print("Running reward for the ", str(num_arms), "arms of the bandit: ", str(total_reward))
        i += 1

print("The agent thinks arm ", str(np.argmax(ww) + 1), " is the most promissing....")

if np.argmax(ww) == np.argmax(-np.array(bandit_arms)):
    print("... and it was right!")
else:
    print("... and it was wrong!")
print("finished")

Running reward for the  4 arms of the bandit:  [1. 0. 0. 0.]
Running reward for the  4 arms of the bandit:  [-2.  2.  2.  5.]
Running reward for the  4 arms of the bandit:  [-5.  1.  1. 12.]
Running reward for the  4 arms of the bandit:  [-7. -3.  0. 17.]
Running reward for the  4 arms of the bandit:  [-5. -8.  2. 20.]
Running reward for the  4 arms of the bandit:  [-5. -7. -1. 30.]
Running reward for the  4 arms of the bandit:  [-9. -4.  1. 37.]
Running reward for the  4 arms of the bandit:  [-9. -5.  0. 45.]
Running reward for the  4 arms of the bandit:  [-7. -5.  5. 54.]
Running reward for the  4 arms of the bandit:  [-13.  -6.   6.  58.]
Running reward for the  4 arms of the bandit:  [-14.  -7.  10.  64.]
Running reward for the  4 arms of the bandit:  [-18.  -8.  17.  72.]
Running reward for the  4 arms of the bandit:  [-21.  -9.  20.  79.]
Running reward for the  4 arms of the bandit:  [-21.  -8.  19.  87.]
Running reward for the  4 arms of the bandit:  [-22.  -8.  19.  94.]
Runni

### 결과
처음에 지정했듯, (+)리워드를 가장 많이 받을 수 있는 -2밴딧이 가장 높게 나왔다.